In [1]:
import flowmap.formats.ugrid
from tvtk.api import tvtk
import geojson
import matplotlib.pyplot as plt
import numpy as np
import shapely.geometry
from tvtk.common import configure_input, configure_source_data
import networkx
import rasterio.mask
import geojson
import flowmap.subgrid
import netCDF4
import tqdm
import pandas as pd
import rasterio.features


import vtk
%matplotlib inline

In [2]:
grid = flowmap.formats.ugrid.UGrid(path='/Users/baart_f/data/rijnland/vanGovert/Leerdam/leerdamwest_map.nc', src_epsg=28992)
dem = flowmap.dem.read_dem('/Users/baart_f/data/rijnland/vanGovert/Leerdam/aw_refi_def_asc.tiff')
crs = geojson.crs.Named(properties={"name": "urn:ogc:def:crs:EPSG::{:d}".format(grid.src_epsg)})

In [3]:
polys = grid.to_polys()

grid->features: 546506it [00:39, 13715.02it/s]


In [5]:
src_epsg = 28992
# convert hull to geojson
crs = geojson.crs.Named(
    properties={
        "name": "urn:ogc:def:crs:EPSG::{:d}".format(src_epsg)
    }
)


In [87]:
nodata = -999
rasterized = rasterio.features.rasterize(
    ((poly, i) for (i, poly) in enumerate(polys)),
    out_shape=dem['band'].shape,
    transform=dem['affine'],
    fill=nodata
)

In [97]:
options = dict(
    dtype=str(rasterized.dtype),
    nodata=nodata,
    count=1,
    compress='lzw',
    tiled=True,
    blockxsize=256,
    blockysize=256,
    driver='GTiff',
    affine=dem['affine'],
    width=dem['width'],
    height=dem['height'],
    crs=rasterio.crs.CRS({'init': 'epsg:%d' % (src_epsg)})

)
with rasterio.open('test.tiff', 'w', **options) as out:
    out.write(rasterized, indexes=1)